<a href="https://colab.research.google.com/github/KrishnaTSasi/FUTURE_DS_01/blob/main/DS_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: Business Sales Dashboard from E-Commerce Data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# 1️ Load dataset
df = pd.read_csv("/content/data.csv", encoding='latin-1')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


Data Cleaning

In [ ]:
# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

In [ ]:
# Remove cancellations (InvoiceNo starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [ ]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Ensure numeric types
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')

In [ ]:
# Create Revenue column
df['Revenue'] = df['Quantity'] * df['UnitPrice']

In [ ]:
# KPIs
total_sales = df['Revenue'].sum()
total_orders = df['InvoiceNo'].nunique()
total_customers = df['CustomerID'].nunique()
avg_order_value = total_sales / total_orders

In [ ]:
print(" KPIs")
print(f"Total Sales (£): {total_sales:,.2f}")
print(f"Total Orders: {total_orders}")
print(f"Total Customers: {total_customers}")
print(f"Average Order Value (£): {avg_order_value:,.2f}")

 KPIs
Total Sales (£): 8,911,407.90
Total Orders: 18536
Total Customers: 4339
Average Order Value (£): 480.76


In [ ]:
# Top Products by Revenue
top_products = df.groupby('Description')['Revenue'].sum().nlargest(10).reset_index()

fig1 = px.bar(top_products,
              x='Revenue',
              y='Description',
              orientation='h',
              title="Top 10 Products by Revenue",
              text='Revenue')
fig1.show()

In [ ]:
# Monthly Sales Trend
df['Month'] = df['InvoiceDate'].dt.to_period('M')
monthly_sales = df.groupby('Month')['Revenue'].sum().reset_index()
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

fig2 = px.line(monthly_sales, x='Month', y='Revenue',
               markers=True,
               title="Monthly Sales Trend")
fig2.show()

In [ ]:
# Revenue by Country
country_sales = df.groupby('Country')['Revenue'].sum().nlargest(10).reset_index()

fig3 = px.pie(country_sales, values='Revenue', names='Country',
              title="Top 10 Countries by Revenue")
fig3.show()

In [ ]:
# Customer Purchase Distribution
customer_spend = df.groupby('CustomerID')['Revenue'].sum().reset_index()

fig4 = px.histogram(customer_spend, x='Revenue', nbins=50,
                    title="Distribution of Customer Spending")
fig4.show()